# Prerequisites: openmm, mdtraj, packmol
- conda config --add channels omnia
- conda install -c omnia openmm
- https://simtk.org/api_docs/openmm/api6_1/python/index.html
- http://docs.openmm.org/7.0.0/userguide/index.html
- http://docs.openmm.org/7.0.0/api-python/index.html
- conda install -c omnia mdtraj
- http://mdtraj.org/1.7.2/
- install packmol with ./configure and make, copy the exe in the working directory
- http://www.ime.unicamp.br/~martinez/packmol/home.shtml

In [1]:
%matplotlib inline
from __future__ import division, print_function
import shutil
from io import StringIO
from math import pi
from scipy import integrate
from IPython.display import display, Math, Latex
import numpy as np
import mdtraj as md
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import os 

In [10]:
WORKDIR = '/home/vidar/playground/'#'/lunarc/nobackup/users/eko12vas/'
os.chdir(WORKDIR)
print("Current directory:", os.getcwd())

# simulation parameters: more to be added

# Dictionary, atom and system properties
d = {
'Forcefield name': 'ff_our',
'Water model': 'spce',
'Box size': 25,
'Input concentrations':    [1.39], 
'Molal concentrations':    [2.0 ],  
'Bond length':             {'SC':0.169500,         'CN':0.115000,             'OH':0.10000}, 
'Harmonic bond constant':  {'SC':252929.5,         'CN':998590.3,             'OH':345000}, 
'Bond angle':              {'SCN':3.13810199508,   'HOH':1.91061193216}, 
'Harmonic angle constant': {'SCN':698.56,          'HOH':383}, 
'Mass':           {'S':32.0660, 'C':12.0110, 'N':14.0067,                          'Na':22.9898, 'K':39.0983, 'Cl':35.453,  'I':126.90 },
'Partial charge': {'S':-0.573,  'C':0.483,   'N':-0.91,   'O':-0.8476, 'H':0.4238, 'Na':1.000,   'K':1.000,   'Cl':-1.000,  'I':-1.000 },  
'Size':           {'S':0.383,   'C':0.335,   'N':0.37,    'O':0.3166,  'H':0,      'Na':0.255,   'K':0.403,   'Cl':0.43900, 'I':0.491  },  
'Well depth':     {'S':1.523,   'C':0.425,   'N':0.310,   'O':0.650,   'H':0,      'Na':0.28,    'K':0.85,    'Cl':0.41600, 'I':0.158  },
'Steps':          {'Simulation':10000, 'Report':1000},
'Constants':      {'A':6.022*pow(10,23),  'Unit converter':pow(10,-9)}}

cation = 'Na'
cation_name = 'sodium'

anion = 'SCN'
anion_name = 'thiocyanate'

def conc2num(conc,box_size):
    n_scn = conc*pow(box_size*d['Constants']['Unit converter'],3)*d['Constants']['A']
    return round(n_scn)

print('Current working directory:', os.getcwd())

Current directory: /home/vidar/playground
Current working directory: /home/vidar/playground


In [11]:
if not os.path.exists('data'):
    os.mkdir('data')

if not os.path.exists('data/'+d['Forcefield name']):
    os.mkdir('data/'+d['Forcefield name'])
    
if not os.path.exists('data/'+d['Forcefield name']+'/'+d['Water model']):
    os.mkdir('data/'+d['Forcefield name']+'/'+d['Water model'])

if not os.path.exists('data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()):
    os.mkdir('data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower())

for conc_m in d['Molal concentrations']:

    if not os.path.exists('data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m'):
        os.mkdir('data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m')

In [12]:
print('Current working directory: ', os.getcwd())
for (conc_input, conc_m) in zip(d['Input concentrations'], d['Molal concentrations']):
    
    wdir = WORKDIR+'data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    
    # write input file for packmol
    
    print('Concentration input:', conc_input, ' M results in', conc2num(conc_input, d['Box size']), 'SCN-ions')
    PACKMOL_PATH = '../packmol/'

    PACKMOL_INPUT = """ 
# Mixture 

tolerance %f
filetype pdb
output %s
add_amber_ter

structure %s
  number %d 
  inside box 0. 0. 0. %f %f %f
end structure
"""

    PACKMOL_INPUT = PACKMOL_INPUT % (1,str(conc_m)+'m_box.pdb',str(conc_m)+'m_scn.pdb',conc2num(conc_input, d['Box size']),d['Box size'],d['Box size'],d['Box size'])
    
    file_handle = open('packmol_input_scn.txt', 'w')
    file_handle.write(PACKMOL_INPUT)
    file_handle.close()
    
    # write pdb file for single SCN

    scn_pdb = """CRYST1  %f  %f  %f  90.00  90.00  90.00 P 1           1
HETATM    1  S   SCN A   1      20.000  20.000  20.000  1.00  0.00           S
HETATM    2  C   SCN A   1      20.000  20.000  21.670  1.00  0.00           C
HETATM    3  N   SCN A   1      20.000  20.000  22.760  1.00  0.00           N
TER       4      SCN A   1   
END"""

    scn_pdb = scn_pdb % (d['Box size'], d['Box size'], d['Box size'])
    
    with open(str(conc_m)+'m_scn.pdb', 'w') as text_file:
        text_file.write(scn_pdb)

    # use packmol to create a system of n_scn randomly placed scn molecules
    os.system("%s < %s" % (PACKMOL_PATH+'packmol', 'packmol_input_scn.txt'))
    
    # move created files to the dedicated folder
    for src_filename in [str(conc_m)+'m_scn.pdb', str(conc_m)+'m_box.pdb', 'packmol_input_scn.txt']:
        dst_filename = os.path.join(wdir, os.path.basename(src_filename))
        shutil.move(src_filename, dst_filename)

Current working directory:  /home/vidar/playground
Concentration input: 1.39  M results in 13 SCN-ions


In [13]:
FFXML_topology = """
<ForceField>

 <AtomTypes>
  <Type name="scn-S" class="SS" element="S" mass="32.0660"/>
  <Type name="scn-C" class="CS" element="C" mass="12.0110"/>
  <Type name="scn-N" class="NS" element="N" mass="14.0067"/>
  <Type name="%s" class="%s" element="%s" mass="%f"/>
  <Type name="spce-O" class="OW" element="O" mass="15.99943"/>
  <Type name="spce-H" class="HW" element="H" mass="1.007947"/>

 </AtomTypes>

 <Residues>
  <Residue name="SCN">
   <Atom name="S" type="scn-S"/>
   <Atom name="C" type="scn-C"/>
   <Atom name="N" type="scn-N"/>
   <Bond from="0" to="1"/>
   <Bond from="1" to="2"/>
  </Residue>

  <Residue name="HOH">
   <Atom name="O" type="spce-O"/>
   <Atom name="H1" type="spce-H"/>
   <Atom name="H2" type="spce-H"/>
   <Bond from="0" to="1"/>
   <Bond from="0" to="2"/>
  </Residue>

  <Residue name="%s">
   <Atom name="%s" type="%s"/>
  </Residue>

 </Residues>"""

FFXML_bonded = """
 <HarmonicBondForce>
  <Bond class1="SS" class2="CS" length="%f" k="%f"/>
  <Bond class1="CS" class2="NS" length="%f" k="%f"/>
  <Bond class1="OW" class2="HW" length="%f" k="%f"/>
 </HarmonicBondForce>

 <HarmonicAngleForce>    
  <Angle class1="SS" class2="CS" class3="NS" angle="%f" k="%f"/>
  <Angle class1="HW" class2="OW" class3="HW" angle="%f" k="%f"/>
 </HarmonicAngleForce>"""

FFXML_nonbonded = """
 <NonbondedForce coulomb14scale="0.5" lj14scale="0.5">
  <Atom type="scn-S" charge="%f" sigma="%f" epsilon="%f"/>
  <Atom type="scn-C" charge="%f" sigma="%f" epsilon="%f"/>
  <Atom type="scn-N" charge="%f" sigma="%f" epsilon="%f"/>
  <Atom type="spce-O" charge="%f" sigma="%f" epsilon="%f"/>
  <Atom type="spce-H" charge="%f" sigma="%f" epsilon="%f"/>
  <Atom type="%s" charge="%f" sigma="%f" epsilon="%f"/>
 </NonbondedForce>

</ForceField>"""

FFXML_topology = FFXML_topology % (cation_name, cation, cation, d['Mass'][cation], cation, cation, cation_name)

FFXML_bonded = FFXML_bonded % (d['Bond length']['SC'], d['Harmonic bond constant']['SC'], d['Bond length']['CN'], d['Harmonic bond constant']['CN'], d['Bond length']['OH'], d['Harmonic bond constant']['OH'], d['Bond angle']['SCN'], d['Harmonic angle constant']['SCN'], d['Bond angle']['HOH'], d['Harmonic angle constant']['HOH'])
            
FFXML_nonbonded = FFXML_nonbonded % (d['Partial charge']['S'], d['Size']['S'], d['Well depth']['S'], d['Partial charge']['C'], d['Size']['C'], d['Well depth']['C'], d['Partial charge']['N'], d['Size']['N'], d['Well depth']['N'], d['Partial charge']['O'], d['Size']['O'], d['Well depth']['O'], d['Partial charge']['H'], d['Size']['H'], d['Well depth']['H'], cation_name, d['Partial charge'][cation], d['Size'][cation], d['Well depth'][cation])

FFXML = FFXML_topology + FFXML_bonded + FFXML_nonbonded

for conc_m in d['Molal concentrations']:
    wdir = WORKDIR+'data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    with open(wdir+str(d['Forcefield name'])+'.xml', 'w') as text_file:
        text_file.write(FFXML)
        
print(FFXML)


<ForceField>

 <AtomTypes>
  <Type name="scn-S" class="SS" element="S" mass="32.0660"/>
  <Type name="scn-C" class="CS" element="C" mass="12.0110"/>
  <Type name="scn-N" class="NS" element="N" mass="14.0067"/>
  <Type name="sodium" class="Na" element="Na" mass="22.989800"/>
  <Type name="spce-O" class="OW" element="O" mass="15.99943"/>
  <Type name="spce-H" class="HW" element="H" mass="1.007947"/>

 </AtomTypes>

 <Residues>
  <Residue name="SCN">
   <Atom name="S" type="scn-S"/>
   <Atom name="C" type="scn-C"/>
   <Atom name="N" type="scn-N"/>
   <Bond from="0" to="1"/>
   <Bond from="1" to="2"/>
  </Residue>

  <Residue name="HOH">
   <Atom name="O" type="spce-O"/>
   <Atom name="H1" type="spce-H"/>
   <Atom name="H2" type="spce-H"/>
   <Bond from="0" to="1"/>
   <Bond from="0" to="2"/>
  </Residue>

  <Residue name="Na">
   <Atom name="Na" type="sodium"/>
  </Residue>

 </Residues>
 <HarmonicBondForce>
  <Bond class1="SS" class2="CS" length="0.169500" k="252929.500000"/>
  <Bond cl

In [14]:
for conc_m in d['Molal concentrations']:
    
    wdir = WORKDIR+'data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    
    openmm_script="""
from __future__ import division, print_function

import numpy as np

# OpenMM Imports
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

print("Current work directory: ", os.getcwd())
pdb = PDBFile("%s")
PDBFile.writeFile(pdb.topology, pdb.positions, open("%s", 'w'))

# load the force field
ff = ForceField("%s")

# modify the topology by adding SC and CN bonds
atoms = list(pdb.topology.atoms())

for i in np.arange(0,len(atoms)-2,3):
    pdb.topology.addBond(atoms[i],atoms[i+1])
    pdb.topology.addBond(atoms[i+1],atoms[i+2])
    i = i + 3


modeller = Modeller(pdb.topology, pdb.positions)
# Add  water and cations
modeller.addSolvent(ff,model='spce',boxSize=(%f,%f,%f)*angstroms,positiveIon='%s')

# Create the OpenMM system
print('Creating OpenMM System')

system = ff.createSystem(modeller.topology,nonbondedMethod=PME,ewaldErrorTolerance=0.0005,
                          nonbondedCutoff=1.0*nanometers, constraints=AllBonds, rigidWater=True)

# system = ff.createSystem(modeller.topology,nonbondedMethod=PME,ewaldErrorTolerance=0.0005,
#                           nonbondedCutoff=1.0*nanometers, constraints=HBonds, rigidWater=True)

# Create the integrator to do Langevin dynamics
integrator = LangevinIntegrator(
                        298*kelvin,       # Temperature of heat bath
                        1.0/picoseconds,  # Friction coefficient
                        2.0*femtoseconds, # Time step
)
integrator.setConstraintTolerance(0.00001)

# NPT ensemble
barostat = MonteCarloBarostat(1.0*bar, 298.0*kelvin, 25) 
system.addForce(barostat)
# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = Platform.getPlatformByName('CPU')

# prop = dict(CudaPrecision='mixed',CudaDeviceIndex='0,1')

# Create the Simulation object
sim = Simulation(modeller.topology, system, integrator, platform) # if prop specified, add ',prop)' to line

# print(platform.getPropertyValue(sim.context))

# Set the particle positions
sim.context.setPositions(modeller.positions)
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*kilojoule/mole, maxIterations=1000)
LocalEnergyMinimizer.minimize(sim.context,tolerance=1*kilojoule/mole,maxIterations=1000)

sim.context.setVelocitiesToTemperature(298*kelvin)

sim.reporters.append(DCDReporter('out.dcd', %d))

sim.reporters.append(StateDataReporter(open('%s', 'w'), %d, step=True,
      potentialEnergy=True, totalEnergy=True, temperature=True, density=True,
          progress=True, remainingTime=True, speed=True, separator='\t', totalSteps = %d))

print('Running Production...')
sim.step(%d)
with open('out.chk', 'wb') as f:
      f.write(sim.context.createCheckpoint())
print('Saving pdb file')
positions = sim.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(sim.topology, positions, open('out.pdb', 'w'))
print('Done!')"""
            
    openmm_script = openmm_script % (wdir+str(conc_m)+'m_box.pdb', str(conc_m)+'m_box.pdb', wdir+d['Forcefield name']+'.xml', d['Box size'], d['Box size'], d['Box size'], cation+'+', d['Steps']['Report'], 'out_file', d['Steps']['Report'], d['Steps']['Simulation'], d['Steps']['Simulation'])
    file_handle = open(wdir+'run.py', 'w')
    file_handle.write(openmm_script)
    file_handle.close()

In [15]:
# Flow control: Set aurora to True if you have access. Else: simulation will be run locally.
aurora = False

if aurora:
    aurora_script="""#!/bin/bash
#SBATCH -p gpu
#SBATCH --exclusive
#SBATCH --gres=gpu:2
#SBATCH --mem-per-cpu=3100
#SBATCH -N 1
#SBATCH -A lu2017-2-5
#
# job time, change for what your job requires
#SBATCH -t 01:00:00
#
# job name
#SBATCH -J scn
#
# filenames stdout and stderr - customise, include %j
#SBATCH -o scn.out
#SBATCH -e scn.err

cd $SLURM_SUBMIT_DIR

#module purge
#module load GCC/5.4.0-2.26
#module load CUDA/8.0.44

module add intelcuda
module unload gcc
module load GCC/4.8.4


python run.py"""
    
    for conc_m in d['Molal concentrations']:
        wdir = WORKDIR+'data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
        with open(wdir+'aurora.sh', 'w') as text_file:
            text_file.write(aurora_script)

In [16]:
for conc_m in d['Molal concentrations']:
    wdir = WORKDIR+'data/'+d['Forcefield name']+'/'+d['Water model']+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    os.chdir(wdir)
    print("Current working directory: ", os.getcwd())
    if aurora:
        !sbatch aurora.sh
        
    else:
        !python run.py
        

Current working directory:  /home/vidar/playground/data/ff_our/spce/nascn/2.0m
Current work directory:  /home/vidar/playground/data/ff_our/spce/nascn/2.0m
Creating OpenMM System
Minimizing energy
Running Production...
Saving pdb file
Done!


In [ ]:
# Plotting activity derivatives as a function of molality

cations = ['Na', 'K']
anions =  ['SCN']
rdfIons = ['C']
colors = ['g', 'b']#, 'b'] #['r', 'b', 'g', 'y', 'c', 'm']
corrIndexes = ['Kruger', 'Vegt']

string = ''

MW_H2O = 18.01528e-3 # kg/mol

for corrIndex in corrIndexes:

    cI = 0

    for cation in cations:

        rI = 0

        for anion in anions:

            if anion is 'SCN':
                os.chdir(WORKDIR+str(d['Box size'])+'Å/'+cation.lower()+anion.lower())

            else:
                os.chdir(WORKDIR+'60Å/'+cation.lower()+anion.lower())

            print('Current working directory: ', os.getcwd())
            # Analysis
            MW_H2O = 18.01528e-3 # kg/mol

            block_range = np.arange(1,19,1)

            molar = np.empty([1, len(d['Molar concentrations'])])
            molal = np.empty([1, len(d['Molal concentrations'])])
            actDer = np.empty([len(block_range),len(d['Molal concentrations'])])
            actDerAvg = np.empty([1, len(d['Molal concentrations'])])
            actDerErr = np.empty([2, len(d['Molal concentrations'])])
            actDer_full = np.empty([1, len(d['Molal concentrations'])])
            extPolVal = np.empty([1, len(d['Molal concentrations'])])

            index = 0
            chargeSet = 1
            rdfIon = rdfIons[rI]

            for (q_s, q_c, q_n) in zip([d['Partial charge']['S'][chargeSet-1]], [d['Partial charge']['C'][chargeSet-1]], [d['Partial charge']['N'][chargeSet-1]]):

                if anion is 'SCN':
                    os.chdir('q_s_'+str(q_s)+'_q_c_'+str(q_c)+'_q_n_'+str(q_n))

                i = 0

                for sigma in [d['Size'][cation]]:

                    j = 0

                    for eps in [d['Well depth'][cation]]:

                        for conc_m in d['Molal concentrations']:                                
                            print("Current working directory: ", os.getcwd(), "\n")

                            if anion is 'SCN':
                                os.chdir('s'+cation+'_'+str(sigma)+'_e'+cation+'_'+str(eps))
                                os.chdir('conc_'+str(conc_m)+'_'+dirs[i,j])
                                print("Current working directory: ", os.getcwd(), "\n")

                            elif anion is 'I':
                                os.chdir('s'+anion+'_'+str(d['Size'][anion])+'_e'+anion+'_'+str(d['Well depth'][anion]))
                                os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                                print("Current working directory: ", os.getcwd(), "\n")

                            else:
                                os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                                print("Current working directory:", os.getcwd(), "\n")

                            cnt = 0
                            ind = 0

                            pdb_full = md.load_pdb('out.pdb')

                            #N_c = len(pdb_full.topology.select('name '+rdfIon)) # Number of N
                            N_an = len(pdb_full.topology.select('name '+rdfIon)) # Number of anions
                            N_cat = len(pdb_full.topology.select('name '+cation)) # Number of cations
                            N_c = len(pdb_full.topology.select('name '+rdfIon+' or name '+cation)) # Number of ions
                            N_w = len(pdb_full.topology.select('name O')) # Number of water molecules

                            g_cc_full = 0
                            g_wc_full = 0
                            V_avg = 0

                            for block in block_range:

                                if os.path.isfile( 'g_cc_'+str(block) ) and os.path.isfile( 'g_wc_'+str(block) ) and os.path.isfile( 'r_'+str(block) ) and os.path.isfile( 'V_'+str(block) ) and os.path.isfile( 'rho_c_'+str(block) ):        # if pickle file exists, use that
                                    print("Loading from saved pickle")

                                    g_cc = np.loadtxt('g_cc_'+str(block))
                                    g_cc_full = g_cc_full+g_cc/len(block_range)
                                    g_wc = np.loadtxt('g_wc_'+str(block))
                                    g_wc_full = g_wc_full+g_wc/len(block_range)
                                    r = np.loadtxt('r_'+str(block))
                                    V = np.loadtxt('V_'+str(block))
                                    V_avg = V_avg + np.loadtxt('V_'+str(block))/len(block_range)
                                    rho_c = np.loadtxt('rho_c_'+str(block))

                                else:
                                    while (cnt < 1):
                                        print("Reading in trajectory...")
                                        sel = pdb_full.topology.select('name O or name '+cation+' or name '+rdfIon)
                                        traj_full = md.load_dcd('out.dcd', top='out.pdb', atom_indices = sel)
                                        print("Number of atoms = ", traj_full.n_atoms)
                                        nbrFrames = traj_full.n_frames
                                        start = 2000
                                        traj_equil = traj_full[start:nbrFrames]
                                        print("Number of frames in equilibrated trajectory: ", traj_equil.n_frames)
                                        cnt = cnt + 1

                                    print('Loading from block '+str(block)+' of trajectory') 

                                    startFrac = start / (d['Steps']['Simulation'] / d['Steps']['Report'])

                                    print('Start fraction: ', startFrac)

                                    endFrac = 1 - startFrac
                                    first = nbrFrames*startFrac + (block-1)*(nbrFrames*endFrac)/len(block_range)
                                    last = nbrFrames*startFrac + block*(nbrFrames*endFrac)/len(block_range)

                                    #first = (block-1)*(nbrFrames*0.5)/len(block_range)+nbrFrames*0.5
                                    #last = block*(nbrFrames*0.5)/len(block_range)+nbrFrames*0.5

                                    traj = traj_full[first:last]

                                    V = 0
                                    for vec in traj.unitcell_lengths:
                                        V = V + vec[0]*vec[1]*vec[2] / traj.n_frames

                                    V_avg = V_avg + V/len(block_range)

                                    #N_c = len(traj.topology.select('name '+cation+' or name '+anion)) # Number of ions
                                    #N_w = len(traj.topology.select('name O')) # Number of water molecules
                                    rho_c = N_c / V # Average number density of ions (nm⁻³)
                                    print("rho_c =",rho_c)
                                    rho_w = N_w / V # Average number density of water molecules (nm⁻³)

                                    # Analysis: Calculating RDFs
                                    r_max = 6.50 #V**(1/3.)/2
                                    #pair_cc = traj.topology.select_pairs('name '+rdfIon, 'name '+rdfIon)
                                    #pair_wc = traj.topology.select_pairs('name O', 'name '+rdfIon)
                                    pair_cc = traj.topology.select_pairs('name '+cation+' or name '+rdfIon, 'name '+cation+' or name '+rdfIon)
                                    pair_wc = traj.topology.select_pairs('name O', 'name '+cation+' or name '+rdfIon)
                                    r, g_cc = md.compute_rdf(traj, pair_cc, r_range=[0,r_max], bin_width=0.01, periodic=True)
                                    r, g_wc = md.compute_rdf(traj, pair_wc, r_range=[0,r_max], bin_width=0.01, periodic=True)
                                    corr = len(pair_cc) / (0.5*N_c**2)# 1-1/N_c # len(pair_cc) / (0.5*N_c**2)
                                    g_cc = g_cc * corr # Re-scaling in order to account for central particle

                                    g_cc_full = g_cc_full+g_cc/len(block_range)
                                    g_wc_full = g_wc_full+g_wc/len(block_range)

                                    for (file_name, var) in zip(['g_cc_'+str(block), 'g_wc_'+str(block), 'r_'+str(block), 'V_'+str(block), 'rho_c_'+str(block)], [g_cc, g_wc, r, V, rho_c]):
                                        if file_name in ('V_'+str(block), 'rho_c_'+str(block)):
                                            f = open( file_name, 'w' )
                                            f.write( repr(var) + '\n' )
                                            f.close()
                                            print("check......")
                                        else: 
                                            np.savetxt(file_name, var)
                                            print("next check...")


                                # Calculating coordination numbers
                                truncInd = 100
                                R = r[100]
                                
                                # Calculating coordination numbers without correction factor
                                if corrIndex is 'Vegt':
                                    dr = r[1]-r[0]
                                    N_cc = rho_c * 4 * pi * np.cumsum( ( g_cc - 1 ) * r ** 2 * dr )
                                    N_wc = rho_c * 4 * pi * np.cumsum( ( g_wc - 1 ) * r ** 2 * dr )
                                    Gamma = N_cc - N_wc
                                
                                # Calculating coordination number with Krüger correction factor
                                else:
                                    dr = r[1]-r[0]
                                    N_ccc = rho_c * 4* pi * np.cumsum( ( g_cc - 1 ) * r ** 2 * (1-3*r/(4*R)+r**3/16*R**3) * dr)
                                    N_wcc = rho_c * 4* pi * np.cumsum( ( g_wc - 1 ) * r ** 2 * (1-3*r/(4*R)+r**3/16*R**3) * dr)
                                
                                
                                '''
                                plt.figure()
                                plt.xlabel('$r$ [nm]')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                plt.plot(r, Gamma, 'r-')
                                plt.yticks( np.arange(-0.7, 0.4, 0.1))
                                plt.ylim((-0.7, 0.4))
                                '''

                                # Accounting for finite size effect on coordination numbers
                                '''
                                Vn = 4*pi/3*r**3 / V
                                ndx=120
                                print('Rconst = ', r[ndx])
                                fixr = True
                                if fixr:
                                    corr_cc = N_c * ( 1 - Vn[ndx] ) / ( N_c * ( 1 - Vn[ndx] ) - N_cc[ndx] - 1 )
                                    corr_wc = N_c * ( 1 - Vn[ndx] ) / ( N_c * ( 1 - Vn[ndx] ) - N_wc[ndx] - 0 )
                                else:
                                    corr_cc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_cc - 1 )
                                    corr_wc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_wc - 0 )

                                g_ccc = g_cc * corr_cc
                                g_wcc = g_wc * corr_wc
                                '''

                                '''
                                #added
                                N_cc = rho_c * 4 * pi * np.cumsum( ( g_ccc - 1 ) * r ** 2 * dr )
                                N_wc = rho_c * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr )
                                corr_cc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_cc - 1 )
                                corr_wc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_wc - 0 )
                                g_ccc = g_cc * corr_cc
                                g_wcc = g_wc * corr_wc
                                #
                                '''
                                Vn = 4*pi/3*r**3 / V
                                
                                # Calculating Vegt correction factors and corresponding rdf:s
                                if corrIndex is 'Vegt':
                                    corr_cc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_cc - 1 )
                                    corr_wc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_wc - 0 )
                                    g_ccc = g_cc * corr_cc 
                                    g_wcc = g_wc * corr_wc
                                    N_ccc = rho_c * 4 * pi * np.cumsum( ( g_ccc - 1 ) * r ** 2 * dr )
                                    N_wcc = rho_c * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr )
                                
                                # Gamma function
                                Gammac = N_ccc - N_wcc 

                                '''
                                plt.figure()
                                plt.xlabel('$r$/nm')
                                plt.ylabel('$N_{cc}$')
                                plt.plot(r, N_ccc, color='red', lw=2)
                                plt.yticks( np.arange(-0.7, 0.4, 0.1))
                                plt.ylim((-0.7, 0.4))
                                plt.title('c = '+str(conc_m)+' m, block '+str(block))

                                plt.figure()
                                plt.xlabel('$r$/nm')
                                plt.ylabel('$N_{wc}$')
                                plt.plot(r, N_wcc, color='red', lw=2)
                                plt.yticks( np.arange(-0.7, 0.4, 0.1))
                                plt.ylim((-0.7, 0.4))
                                plt.title('c = '+str(conc_m)+' m, block '+str(block))

                                plt.figure()
                                plt.xlabel('$r$/nm')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                plt.plot(r, Gamma, color='red', lw=2, label='uncorrected')
                                plt.plot(r, Gammac, color='green', lw=2, label='corrected')
                                plt.legend(loc=0, frameon=False, fontsize=16)
                                plt.yticks( np.arange(-0.7, 0.4, 0.1))
                                plt.ylim((-0.7, 0.4))
                                plt.title('c = '+str(conc_m)+' m, block '+str(block))
                                '''
                                
                                if corrIndex is 'Vegt':
                                    Gammac_plateau = Gammac[truncInd]
                                    
                                else:
                                    Gammac_plateau = Gammac[truncInd*2]
                                
                                #print("g_cc =", g_ccc)
                                #print("g_wc =", g_wc)
                                #print("Preferential binding parameter =",Gammac_plateau, "\n")
                                #print("Average salt density =", rho_c)
                                #print("Salt density at periodic boundary around salt =", rho_cc)
                                #print("Salt density at periodic boundary around water =", rho_wc, "\n")

                                actDer[ind, index] = 1 / ( 1 + Gammac_plateau )

                                ind = ind + 1

                            '''
                            if os.path.isfile( 'g_cc' ) and os.path.isfile( 'g_wc' ):
                                print("Loading full RDFs...")
                                g_cc = np.loadtxt('g_cc')
                                g_wc = np.loadtxt('g_wc')
                                r = np.loadtxt('r')

                            else:
                                r_max = V_avg**(1/3.)/2
                                pair_cc = traj_equil.topology.select_pairs('name '+cation+' or name C', 'name '+cation+' or name C')
                                pair_wc = traj_equil.topology.select_pairs('name O', 'name '+cation+' or name C')
                                r, g_cc = md.compute_rdf(traj_equil, pair_cc, r_range=[0,r_max], bin_width=0.01, periodic=True)
                                r, g_wc = md.compute_rdf(traj_equil, pair_wc, r_range=[0,r_max], bin_width=0.01, periodic=True)
                                corr = len(pair_cc) / (0.5*N_c**2)# 1-1/N_c # len(pair_cc) / (0.5*N_c**2)
                                g_cc = g_cc * corr # Re-scaling in order to account for central particle

                                for (file_name, var) in zip(['g_cc', 'g_wc', 'r'], [g_cc, g_wc, r]):
                                    np.savetxt(file_name, var)
                                    print("next check...")
                            '''
                            rho_c = N_c / V_avg # Average number density of ions (nm⁻³)
                            print("rho_c =",rho_c)
                            rho_w = N_w / V_avg # Average number density of water molecules (nm⁻³)

                            # Calculating coordination numbers without correction factor, full trajectory
                            if corrIndex is 'Vegt':
                                dr = r[1]-r[0]
                                N_cc = rho_c * 4 * pi * np.cumsum( ( g_cc_full - 1 ) * r ** 2 * dr )
                                N_wc = rho_c * 4 * pi * np.cumsum( ( g_wc_full - 1 ) * r ** 2 * dr )
                                Gamma_full = N_cc - N_wc
                            
                            # Calculating coordination numbers with Kkrüger correction factor, full trajectory
                            else:
                                truncIndLow = 0
                                truncIndHigh = 101
                                IndRange = np.arange(truncIndLow, truncIndHigh)
                                #correction = np.empty([1, truncIndHigh])
                                N_ccc_Kruger = np.empty([1, truncIndHigh])
                                N_wcc_Kruger = np.empty([1, truncIndHigh])
                                Gammac_full_Kruger = np.empty([1, truncIndHigh])
                                R = np.empty([1, truncIndHigh])
                                
                                for Ind in IndRange:
                                    R[0, Ind] = r[Ind]
                                    dr = r[1]-r[0]
                                    correction = 1-3*r/(4*R[0, Ind])+r**3/(16*R[0, Ind])
                                    N_ccc = rho_c * 4 * pi * np.cumsum( ( g_cc_full - 1) * r ** 2 * correction * dr)
                                    N_wcc = rho_c * 4 * pi * np.cumsum( ( g_wc_full - 1) * r ** 2 * correction * dr)
                                    #print("Check: ", N_ccc[:])
                                    N_ccc_Kruger[0, Ind] = N_ccc[2*Ind]
                                    N_wcc_Kruger[0, Ind] = N_wcc[2*Ind]
                                    Gammac_full_Kruger[0, Ind] = N_ccc_Kruger[0, Ind] - N_wcc_Kruger[0, Ind]
                                    print("correction: ", correction[Ind])
                                    
                                plt.figure()
                                plt.plot(r, N_ccc-N_wcc, 'm-', label='corrected with '+corrIndex+', '+cation+anion)
                                plt.plot(r, N_cc-N_wc, 'g-', label='uncorrected, '+cation+anion)
                                plt.xlabel('r [nm]')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                plt.legend(loc=0, frameon=False, fontsize=9)
                                
                                '''
                                plt.figure()
                                plt.plot(r, Gammac_full_Kruger[0,Ind])
                                plt.xlabel('r [nm]')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                ''' 
                                print("Array check: ", Gammac_full_Kruger[0,Ind])
                                print("N_ccc =", N_ccc_Kruger[0,Ind])
                                print("N_wcc =", N_wcc_Kruger[0,Ind])
                                
                                plt.figure()
                                plt.xlabel('$1/R$ [nm$^{-1}$]')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                plt.plot(1/R[0,:], Gammac_full_Kruger[0,:])
                                plt.xlim((0.0, 0.7))

                                deltaX = 1/R[0,-1]-1/R[0,-1-10]
                                deltaY = Gammac_full_Kruger[0,-1] - Gammac_full_Kruger[0,-1-10]
                                slope = deltaY/deltaX
                                extPolVal[0,index] = Gammac_full_Kruger[0,-1]-slope*(1/R[0,-1])
                                print("Last value of R: ", R[0,-1])
                                print("Last value of R, resulting in a 1/R of: ", 1/R[0,-1])
                                print("10th last value of R, resulting in a 1/R of: ", 1/R[0,-1-10])
                                print("Delta x = ", deltaX)
                                print("Last value of R, resulting in a gamma function of: ", Gammac_full_Kruger[0,-1])
                                print("10th last value of R, resulting in a gamma function of: ", Gammac_full_Kruger[0,-1-10])
                                print("Delta y = ", deltaY)
                                print("Obtained value, gamma function: ", extPolVal[0,index])
                            
                            Vn = 4*pi/3*r**3 / V_avg
                            
                            # Calculating Vegt correction factors and corresponding rdf:s, full trajectory
                            if corrIndex is 'Vegt':
                                corr_cc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_cc - 1 )
                                corr_wc = N_c * ( 1 - Vn ) / ( N_c * ( 1 - Vn ) - N_wc - 0 )
                                g_ccc = g_cc_full * corr_cc
                                g_wcc = g_wc_full * corr_wc
                                #np.savetxt('excCoordSimNoCorr', Gamma_full)
                                N_ccc = rho_c * 4 * pi * np.cumsum( ( g_ccc - 1 ) * r ** 2 * dr)
                                N_wcc = rho_c * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr)
                                Gammac_full = N_ccc - N_wcc
                                Gammac_plateau_full = Gammac_full[truncInd]
                                actDer_full[0, index] = 1 / ( 1 + Gammac_plateau_full )
                                plt.figure()
                                plt.plot(r, N_ccc-N_wcc, 'm-', label='corrected with '+corrIndex+', '+cation+anion)
                                plt.plot(r, N_cc-N_wc, 'g-', label='uncorrected, '+cation+anion)
                                plt.xlabel('r [nm]')
                                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                                plt.legend(loc=0, frameon=False, fontsize=9)
                            
                            else:
                                Gammac_plateau_full = Gammac_full_Kruger[0, truncInd]
                                actDer_full[0, index] = 1 / ( 1 + Gammac_plateau_full )
                                #actDer_full[0, index] = 1 / ( 1 + extPolVal[0, index])
                            
                            #corr_cc = corr_cc_func[101]
                            #corr_wc = corr_wc_func[101]

                            
                            #rho_cc = (N_c-(N_c*Vn[100]+N_cc[100]+1))/(V*(1-Vn[100]))
                            #rho_wc = (N_c-(N_c*Vn[100]+N_wc[100]+0))/(V*(1-Vn[100]))

                            
                            plt.figure()
                            plt.xlabel('$r$ [nm]')
                            plt.ylabel('$g(r)$')
                            plt.plot(r, g_cc_full, 'm-', label='ion around ion, '+cation+anion)
                            plt.plot(r, g_wc_full, 'g-', label='ion around water, '+cation+anion)
                            #plt.plot(r, g_wcc, 'b-', label=rdfIon+' around O')
                            plt.legend(loc=0, frameon=False, fontsize=16)
                            #plt.xticks( np.arange(0.1, 0.7, 0.1))
                            plt.axis([0,6.5,0,2.5])
                            plt.savefig('rdf_'+rdfIon)
                            plt.show()

                            #N_ccc = rho_c * 4 * pi * np.cumsum( ( g_ccc - 1 ) * r ** 2 * dr )
                            #N_wcc = rho_c * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr )
                            #Gammac_full = N_ccc - N_wcc 
                            #Gammac_plateau_full = np.mean(Gammac_full[95:115])
                            #actDer_full[0, index] = 1 / ( 1 + Gammac_plateau_full )

                            #for (file_name, var) in zip(['excCoordSim'+corrIndex], [Gammac_full]):
                            #    np.savetxt(file_name, var)
                            
                            '''
                            plt.figure()
                            plt.xlabel('$r$/nm')
                            plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                            plt.plot(r, Gamma_full, color='red', lw=2, label='uncorrected')
                            plt.plot(r, Gammac_full, color='blue', lw=2, label='corrected')
                            plt.legend(loc=0, frameon=False, fontsize=16)
                            plt.yticks( np.arange(-0.7, 0.4, 0.1))
                            plt.ylim((-0.7, 0.4))
                            plt.xlim((0, 2.7))
                            plt.title('c = '+str(conc_m)+' m, full trajectory')
                            '''

                            molar[0, index] = N_c/(2*d['Constants']['A']*V_avg*1e-24)
                            molal[0, index] = N_c/(2*N_w*MW_H2O)

                            actDerAvg[0, index] = np.mean(actDer[:, index])
                            #actDerErr[0, index] = actDerAvg[0, index] - np.min(actDer[:, index])
                            #actDerErr[1, index] = np.max(actDer[:, index]) - actDerAvg[0, index]
                            actDerErr[0, index] = np.std(actDer[:, index])
                            actDerErr[1, index] = np.std(actDer[:, index])

                            print("Propertites for", molar[0, index], "M:")
                            print("Number of ions =", N_c)
                            print("Number of anions =", N_an)
                            print("Number of cations =", N_cat)
                            print("Number of water molecules =", N_w)
                            print("Molality =", molal[0,index], "m")
                            print("Average box side length =", V_avg**(1/3), "nm")
                            print("Average box volume =", V_avg, "nm³")
                            print("Activity derivatives: ", actDer[:, index], '\n')
                            print("Activity derivative, whole trajectory: ", actDer_full[0, index])
                            
                            index = index + 1

                            os.chdir('..')

                            if anion is 'SCN' or anion is 'I':
                                os.chdir('..')
                        
                        j = j+1
                    i = i+1

            for (file_name, var) in zip(['actDerSim'+corrIndex, 'actDerErrSim'+corrIndex], [actDer_full[0,:], actDerErr[:,:]]):
                np.savetxt(file_name, var)

            print("Current working directory: ", os.getcwd())
            #print("Activity derivatives =", actDer)
            #print("Molalities =", c_m)

            
            # Loading experimental data
            m = np.loadtxt('molality', delimiter=',', unpack=True)
            a = np.loadtxt('actDer', delimiter=',', unpack=True)
            mShort = m[0:360]
            aShort = a[0:360]


            plt.xlabel("$m$", fontsize=17)
            plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)
            plt.plot(mShort, aShort, color=colors[cI], lw=2, label=cation+anion+', from experiment')


            #print("Activity derivative averages: ", actDerAvg)

            plt.errorbar(molal[0,:], actDer_full[0,:], actDerErr[:,:], fmt=colors[cI]+'^', label=cation+anion+', from simulation', alpha=0.6)
            plt.legend(loc=0, frameon=False, fontsize=9)
            plt.xlim((0, 3.5))
            plt.ylim((0.7, 2.0))
            

            #plt.plot(molal[0 , :], actDerAvg[0, :], 'ro', label='Simulation')
            #for c in np.arange(0, len(conc_range), 1):
            #    plt.errorbar(molal[:, c], actDer[:, c], lw=1, capsize=5, capthick=2, color='r', fmt='-')

            cI = cI + 1
            rI = rI + 1

            os.chdir('..')

plt.savefig('rdf_'+str(cation)+str(anion)+'.png')#, bbox_inches='tight', pad_inches=0.3)
#plt.savefig('firstCorr', bbox_inches='tight', pad_inches=0.1)
plt.show()

print("Current working directory: ", os.getcwd())    

In [ ]:
# Plotting activity derivatives as a function of molality

cations = ['Na', 'K']
anions =  ['SCN']
rdfIons = ['C']
colors = ['g', 'b'] #['r', 'b', 'g', 'y', 'c', 'm']
corrIndexes = ['Vegt', 'Kruger']

actDerErrSimVegt = np.empty(0) 
actDerErrSimKruger = np.empty(0)

string = ''

MW_H2O = 18.01528e-3 # kg/mol

for tickInd in np.arange(0,2,1):

    cI = 0

    for cation in cations:

        rI = 0

        for anion in anions:

            index = 0

            molar = np.empty([1, len(d['Molar concentrations'])])
            molal = np.empty([1, len(d['Molal concentrations'])])

            if anion is 'SCN':
                os.chdir(WORKDIR+str(d['Box size'])+'Å/'+cation.lower()+anion.lower())

            else:
                os.chdir(WORKDIR+'60Å/'+cation.lower()+anion.lower())

            print('Current working directory: ', os.getcwd())
            # Analysis
            MW_H2O = 18.01528e-3 # kg/mol

            block_range = np.arange(1,19,1)

            index = 0
            rdfIon = rdfIons[rI]

            i = 0

            if anion is 'SCN':
                os.chdir('q_s_'+str(d['Partial charge']['S'][tickInd])+'_q_c_'+str(d['Partial charge']['C'][tickInd])+'_q_n_'+str(d['Partial charge']['N'][tickInd]))

            sigma = d['Size'][cation][tickInd]

            j = 0

            eps = d['Well depth'][cation][tickInd]

            for conc_m in d['Molal concentrations']:                                
                print("Current working directory: ", os.getcwd(), "\n")

                if anion is 'SCN':

                    os.chdir('s'+cation+'_'+str(sigma)+'_e'+cation+'_'+str(eps))
                    os.chdir('conc_'+str(conc_m)+'_sS_'+str(d['Size']['S'][tickInd])+'_eS_'+str(d['Well depth']['S'][tickInd])+'_sC_'+str(d['Size']['C'][tickInd])+'_eC_'+str(d['Well depth']['C'][tickInd])+'_sN_'+str(d['Size']['N'][tickInd])+'_eN_'+str(d['Well depth']['N'][tickInd]))
                    #os.chdir('conc_'+str(conc_m)+'_'+dirs[i,j])
                    print("Current working directory: ", os.getcwd(), "\n")


                elif anion is 'I':
                    os.chdir('s'+anion+'_'+str(d['Size'][anion])+'_e'+anion+'_'+str(d['Well depth'][anion]))
                    os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                    print("Current working directory: ", os.getcwd(), "\n")

                else:
                    os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                    print("Current working directory:", os.getcwd(), "\n")

                cnt = 0
                ind = 0

                pdb_full = md.load_pdb('out.pdb')
                N_c = len(pdb_full.topology.select('name '+rdfIon+' or name '+cation)) # Number of ions
                N_w = len(pdb_full.topology.select('name O')) # Number of water molecules

                V_avg = 0

                for block in block_range:
                    V = np.loadtxt('V_'+str(block))
                    V_avg = V_avg + np.loadtxt('V_'+str(block))/len(block_range)

                Gamma_full = np.loadtxt('excCoordSimNoCorr')

                for corrIndex in corrIndexes:
                    Gammac_full_Vegt = np.loadtxt('excCoordSimVegt')
                    Gammac_full_Kruger = np.loadtxt('excCoordSimKruger')

                r = np.loadtxt('r_10')
                r = r[0:len(Gammac_full_Kruger)]

                '''
                plt.figure()
                plt.xlabel('$r$/nm')
                plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                plt.plot(r, Gamma_full, color='red', lw=2, label='No correction')
                plt.plot(r, Gammac_full_Vegt, color='blue', lw=2, label='Vegt correction')
                plt.plot(r, Gammac_full_Kruger, color='green', lw=2, label='Kruger correction')
                plt.legend(loc=0, frameon=False, fontsize=16)
                plt.yticks( np.arange(-0.7, 0.4, 0.1))
                plt.ylim((-0.7, 0.4))
                plt.xlim((0, 2.7))
                plt.title('c = '+str(conc_m)+' m, full trajectory')
                '''

                molar[0, index] = N_c/(2*d['Constants']['A']*V_avg*1e-24)
                molal[0, index] = N_c/(2*N_w*MW_H2O)

                index = index + 1

                if anion is 'Cl':
                    os.chdir('..')

                else: 
                    os.chdir('../..')

                    j = j+1
                i = i+1     


            for corrIndex in corrIndexes:
                actDerSimVegt = np.loadtxt('actDerSimVegt')
                actDerErrSimVegt= np.loadtxt('actDerErrSimVegt')
                actDerSimKruger = np.loadtxt('actDerSimKruger')
                actDerErrSimKruger = np.loadtxt('actDerErrSimKruger')

                #print("Activity derivatives, Vegt: ", actDerSimVegt[0:5])


            #fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

            #axes[0].plot(mShort, aShort, color=colors[cI], lw=2, label=cation+anion+' from experiment')
            #axes[0].errorbar(molal[0,:], actDerSimVegt, actDerErrSimVegt, fmt=colors[cI]+'^', label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

            m = np.loadtxt('molality', delimiter=',', unpack=True)
            a = np.loadtxt('actDer', delimiter=',', unpack=True)
            mShort = m[0:360]
            aShort = a[0:360]

            #print('Activity derivatives, Vegt:', actDerSimVegt[0:5])
            #print('Activity derivative errors, Vegt:', actDerErrSimVegt[0:5])
            print('Molalities:', molal[0,:])

            if tickInd == 0 and cation is 'Na' and anion is 'SCN':
                salt1conc = molal[0,:]
                salt1actVegt = np.loadtxt('actDerSimVegt')
                salt1actErrVegt = np.loadtxt('actDerErrSimVegt')
                salt1actKruger = np.loadtxt('actDerSimKruger')
                salt1actErrKruger = np.loadtxt('actDerErrSimKruger')
                salt1m = mShort
                salt1a = aShort


            elif tickInd == 0 and cation is 'K' and anion is 'SCN':
                salt2conc = molal[0,:]
                salt2actVegt = np.loadtxt('actDerSimVegt')
                salt2actErrVegt = np.loadtxt('actDerErrSimVegt')
                salt2actKruger = np.loadtxt('actDerSimKruger')
                salt2actErrKruger = np.loadtxt('actDerErrSimKruger')
                salt2m = mShort
                salt2a = aShort


            elif tickInd == 1 and cation is 'Na' and anion is 'SCN':
                salt3conc = molal[0,:]
                salt3actVegt = np.loadtxt('actDerSimVegt')
                salt3actErrVegt = np.loadtxt('actDerErrSimVegt')
                salt3actKruger = np.loadtxt('actDerSimKruger')
                salt3actErrKruger = np.loadtxt('actDerErrSimKruger')
                salt3m = mShort
                salt3a = aShort

            elif tickInd == 1 and cation is 'K' and anion is 'SCN':
                salt4conc = molal[0,:]
                salt4actVegt = np.loadtxt('actDerSimVegt')
                salt4actErrVegt = np.loadtxt('actDerErrSimVegt')
                salt4actKruger = np.loadtxt('actDerSimKruger')
                salt4actErrKruger = np.loadtxt('actDerErrSimKruger')
                salt4m = mShort
                salt4a = aShort


            #plt.xlabel("$m$", fontsize=17)
            #plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)

            #plt.plot(mShort, aShort, color=colors[cI], lw=2, label=cation+anion+' from experiment')
            #plt.errorbar(molal[0,:], actDerSimVegt, actDerErrSimVegt, fmt=colors[cI]+'^', label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
            #plt.errorbar(molal[0,:], actDerSimKruger, actDerErrSimKruger, fmt=colors[cI]+'s', label=cation+anion+' from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha=0.6)
            #plt.plot(molal[0,:], actDerSimVegt, colors[cI]+'^', label=cation+anion+', from simulation, Vegt', alpha=0.6)
            #plt.plot(molal[0,:], actDerSimKruger, colors[cI]+'s', label=cation+anion+', from simulation, Krüger', alpha=0.6)

            #plt.legend(loc='upper left', frameon=False, fontsize=9)
            #plt.xlim((0, 3.5))
            #plt.ylim((0.7, 2.0))

            cI = cI + 1
            rI = rI +1

print("Salt activities: ", salt1actErrVegt, salt2actErrVegt, salt3actErrVegt, salt4actErrVegt)
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

axes[0].plot(salt3m, salt3a, color=colors[0], lw=2, label='NaSCN from experiment') #label=cation+anion+' from experiment')
axes[0].plot(salt4m, salt4a, color=colors[1], lw=2, label='KSCN from experiment') #label=cation+anion+' from experiment')
axes[0].errorbar(salt3conc, salt3actVegt, salt3actErrVegt, fmt=colors[0]+'^', label='NaSCN from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[0].errorbar(salt4conc, salt4actVegt, salt4actErrVegt, fmt=colors[1]+'^', label='KSCN from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[0].errorbar(salt3conc, salt3actKruger, salt3actErrKruger, fmt=colors[0]+'s', label='NaSCN from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[0].errorbar(salt4conc, salt4actKruger, salt4actErrKruger, fmt=colors[1]+'s', label='KSCN from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

axes[0].set_xlabel("$m$", fontsize=17)
axes[0].set_ylabel(r'$a_{c}^{\prime}$', fontsize=17)
plt.sca(axes[0])
#plt.xticks(x, labels, color='k')
#plt.yticks(np.arange(-.6,.1,0.2),color='k')
axes[0].set_xlim(0,3.5)
axes[0].set_ylim(0.5,2.0)
axes[0].legend(frameon=False,numpoints=1,loc='upper left', fontsize=10)

axes[1].plot(salt1m, salt1a, color=colors[0], lw=2, label='NaSCN from experiment') #label=cation+anion+' from experiment')
axes[1].plot(salt2m, salt2a, color=colors[1], lw=2, label='KSCN from experiment') #label=cation+anion+' from experiment')
axes[1].errorbar(salt1conc, salt1actVegt, salt1actErrVegt, fmt=colors[0]+'^', label='NaSCN from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6) #label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[1].errorbar(salt2conc, salt2actVegt, salt2actErrVegt, fmt=colors[1]+'^', label='KSCN from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6) #label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[1].errorbar(salt1conc, salt1actKruger, salt1actErrKruger, fmt=colors[0]+'s', label='NaSCN from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[1].errorbar(salt2conc, salt2actKruger, salt2actErrKruger, fmt=colors[1]+'s', label='KSCN from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

axes[1].set_xlabel("$m$", fontsize=17)
plt.sca(axes[1])
#plt.xticks(x, labels, color='k')
#plt.yticks(np.arange(-.6,.1,0.2),color='k')
axes[1].set_xlim(0,3.5)
#axes[1].set_ylim(0.5,1.8)
axes[1].legend(frameon=False,numpoints=1,loc='upper left', fontsize=10)

axes[0].text(3.0,1.8,'a',size=55)
axes[1].text(3.0,1.8,'b',size=55)
'''
plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)
plt.legend(loc='upper left', frameon=False, fontsize=9)
plt.xlim((0, 3.5))
plt.ylim((0.7, 2.0))
'''

'''
plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)
plt.legend(loc='upper left', frameon=False, fontsize=9)
plt.xlim((0, 3.5))
plt.ylim((0.7, 2.0))
'''

if anion is 'SCN':
    os.chdir('../../figs')

else:
    os.chdir('../figs')
        
print("Activity derivative error:", actDerErrSimKruger)
print("Current working directory: ", os.getcwd())
fig.tight_layout()
plt.savefig('actDersHessKrügerAll.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show

In [ ]:
# Plotting activity derivatives as a function of molality

cations = ['Na', 'K']
anions =  ['I', 'Cl']
rdfIons = ['I', 'Cl']
colors = ['g', 'b'] #['r', 'b', 'g', 'y', 'c', 'm']
corrIndexes = ['Vegt', 'Kruger']

actDerErrSimVegt = np.empty(0) 
actDerErrSimKruger = np.empty(0)

string = ''

MW_H2O = 18.01528e-3 # kg/mol

for chargeSet in np.arange(1,2,1):

    cI = 0

    for cation in cations:

        rI = 0

        for anion in anions:

            index = 0

            molar = np.empty([1, len(d['Molar concentrations'])])
            molal = np.empty([1, len(d['Molal concentrations'])])

            if anion is 'SCN':
                os.chdir(WORKDIR+str(d['Box size'])+'Å/'+cation.lower()+anion.lower())

            else:
                os.chdir(WORKDIR+'60Å/'+cation.lower()+anion.lower())

            print('Current working directory: ', os.getcwd())
            # Analysis
            MW_H2O = 18.01528e-3 # kg/mol

            block_range = np.arange(1,19,1)

            index = 0
            rdfIon = rdfIons[rI]

            for (q_s, q_c, q_n) in zip([d['Partial charge']['S'][chargeSet-1]], [d['Partial charge']['C'][chargeSet-1]], [d['Partial charge']['N'][chargeSet-1]]):

                i = 0

                if anion is 'SCN':
                    os.chdir('q_s_'+str(q_s)+'_q_c_'+str(q_c)+'_q_n_'+str(q_n))

                for sigma in [d['Size'][cation]]:

                    j = 0

                    for eps in [d['Well depth'][cation]]:

                        for conc_m in d['Molal concentrations']:                                
                            print("Current working directory: ", os.getcwd(), "\n")

                            if anion is 'SCN' and chargeSet == 0:
                                
                                os.chdir('s'+cation+'_'+str(sigma)+'_e'+cation+'_'+str(eps))
                                #os.chdir('conc_'+str(conc_m)+'_'+dirs[i,j])
                                print("Current working directory: ", os.getcwd(), "\n")
                                         

                            elif anion is 'I':
                                os.chdir('s'+anion+'_'+str(d['Size'][anion])+'_e'+anion+'_'+str(d['Well depth'][anion]))
                                os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                                print("Current working directory: ", os.getcwd(), "\n")

                            else:
                                os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                                print("Current working directory:", os.getcwd(), "\n")

                            cnt = 0
                            ind = 0

                            pdb_full = md.load_pdb('out.pdb')
                            N_c = len(pdb_full.topology.select('name '+rdfIon+' or name '+cation)) # Number of ions
                            N_w = len(pdb_full.topology.select('name O')) # Number of water molecules

                            V_avg = 0

                            for block in block_range:
                                V = np.loadtxt('V_'+str(block))
                                V_avg = V_avg + np.loadtxt('V_'+str(block))/len(block_range)

                            Gamma_full = np.loadtxt('excCoordSimNoCorr')

                            for corrIndex in corrIndexes:
                                Gammac_full_Vegt = np.loadtxt('excCoordSimVegt')
                                Gammac_full_Kruger = np.loadtxt('excCoordSimKruger')

                            r = np.loadtxt('r_10')
                            r = r[0:len(Gammac_full_Kruger)]

                            '''
                            plt.figure()
                            plt.xlabel('$r$/nm')
                            plt.ylabel('$\\Gamma = N_{cc}-N_{wc}$')
                            plt.plot(r, Gamma_full, color='red', lw=2, label='No correction')
                            plt.plot(r, Gammac_full_Vegt, color='blue', lw=2, label='Vegt correction')
                            plt.plot(r, Gammac_full_Kruger, color='green', lw=2, label='Kruger correction')
                            plt.legend(loc=0, frameon=False, fontsize=16)
                            plt.yticks( np.arange(-0.7, 0.4, 0.1))
                            plt.ylim((-0.7, 0.4))
                            plt.xlim((0, 2.7))
                            plt.title('c = '+str(conc_m)+' m, full trajectory')
                            '''

                            molar[0, index] = N_c/(2*d['Constants']['A']*V_avg*1e-24)
                            molal[0, index] = N_c/(2*N_w*MW_H2O)

                            index = index + 1

                            if anion is 'Cl':
                                os.chdir('..')

                            else: 
                                os.chdir('../..')

                        j = j+1
                    i = i+1     


                for corrIndex in corrIndexes:
                    actDerSimVegt = np.loadtxt('actDerSimVegt')
                    actDerErrSimVegt= np.loadtxt('actDerErrSimVegt')
                    actDerSimKruger = np.loadtxt('actDerSimKruger')
                    actDerErrSimKruger = np.loadtxt('actDerErrSimKruger')

                    print("Activity derivatives, Vegt: ", actDerSimVegt[0:5])


                #fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

                #axes[0].plot(mShort, aShort, color=colors[cI], lw=2, label=cation+anion+' from experiment')
                #axes[0].errorbar(molal[0,:], actDerSimVegt, actDerErrSimVegt, fmt=colors[cI]+'^', label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

                m = np.loadtxt('molality', delimiter=',', unpack=True)
                a = np.loadtxt('actDer', delimiter=',', unpack=True)
                mShort = m[0:360]
                aShort = a[0:360]

                #print('Activity derivatives, Vegt:', actDerSimVegt[0:5])
                #print('Activity derivative errors, Vegt:', actDerErrSimVegt[0:5])
                print('Molalities:', molal[0,:])

                if cation is 'Na' and anion is 'I':
                    salt1conc = molal[0,:]
                    salt1act = np.loadtxt('actDerSimVegt')
                    salt1actErr = np.loadtxt('actDerErrSimVegt')
                    salt1m = mShort
                    salt1a = aShort
                    

                elif cation is 'K' and anion is 'I':
                    salt2conc = molal[0,:]
                    salt2act = np.loadtxt('actDerSimVegt')
                    salt2actErr = np.loadtxt('actDerErrSimVegt')
                    salt2m = mShort
                    salt2a = aShort
                
                
                elif cation is 'Na' and anion is 'Cl':
                    salt3conc = molal[0,:]
                    salt3act = np.loadtxt('actDerSimVegt')
                    salt3actErr = np.loadtxt('actDerErrSimVegt')
                    salt3m = mShort
                    salt3a = aShort

                elif cation is 'K' and anion is 'Cl':
                    salt4conc = molal[0,:]
                    salt4act = np.loadtxt('actDerSimVegt')
                    salt4actErr = np.loadtxt('actDerErrSimVegt')
                    salt4m = mShort
                    salt4a = aShort
                
                
                #plt.xlabel("$m$", fontsize=17)
                #plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)

                #plt.plot(mShort, aShort, color=colors[cI], lw=2, label=cation+anion+' from experiment')
                #plt.errorbar(molal[0,:], actDerSimVegt, actDerErrSimVegt, fmt=colors[cI]+'^', label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
                #plt.errorbar(molal[0,:], actDerSimKruger, actDerErrSimKruger, fmt=colors[cI]+'s', label=cation+anion+' from simulation (correction from Krüger $\it{et}$ $\it{al.}$)', alpha=0.6)
                #plt.plot(molal[0,:], actDerSimVegt, colors[cI]+'^', label=cation+anion+', from simulation, Vegt', alpha=0.6)
                #plt.plot(molal[0,:], actDerSimKruger, colors[cI]+'s', label=cation+anion+', from simulation, Krüger', alpha=0.6)

                #plt.legend(loc='upper left', frameon=False, fontsize=9)
                #plt.xlim((0, 3.5))
                #plt.ylim((0.7, 2.0))

                cI = cI + 1
                rI = rI +1

#print("Salt activities: ", salt1act, salt2act, salt3act, salt4act)
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

axes[0].plot(salt1m, salt1a, color=colors[0], lw=2, label='NaI from experiment') #label=cation+anion+' from experiment')
axes[0].plot(salt2m, salt2a, color=colors[1], lw=2, label='KI from experiment') #label=cation+anion+' from experiment')
axes[0].errorbar(salt1conc, salt1act, salt1actErr, fmt=colors[0]+'^', label='NaI from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[0].errorbar(salt2conc, salt2act, salt2actErr, fmt=colors[1]+'^', label='KI from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6)#label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

axes[0].set_ylabel(r'$a_{c}^{\prime}$', fontsize=17)
axes[0].set_xlabel("$m$", fontsize=17)
plt.sca(axes[0])
#plt.xticks(x, labels, color='k')
#plt.yticks(np.arange(-.6,.1,0.2),color='k')
axes[0].set_xlim(0,3.5)
axes[0].set_ylim(0.5,2.0)
axes[0].legend(frameon=False,numpoints=1,loc='upper left', fontsize=10)

axes[1].plot(salt3m, salt3a, color=colors[0], lw=2, label='NaCl from experiment') #label=cation+anion+' from experiment')
axes[1].plot(salt4m, salt4a, color=colors[1], lw=2, label='KCl from experiment') #label=cation+anion+' from experiment')
axes[1].errorbar(salt3conc, salt3act, salt3actErr, fmt=colors[0]+'^', label='NaCl from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6) #label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)
axes[1].errorbar(salt4conc, salt4act, salt4actErr, fmt=colors[1]+'^', label='KCl from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha = 0.6) #label=cation+anion+' from simulation (correction from Hess $\it{et}$ $\it{al.}$)', alpha=0.6)

axes[1].set_xlabel("$m$", fontsize=17)
plt.sca(axes[0])
#plt.xticks(x, labels, color='k')
#plt.yticks(np.arange(-.6,.1,0.2),color='k')
axes[1].set_xlim(0,3.5)
axes[1].legend(frameon=False,numpoints=1,loc='upper left', fontsize=10)

axes[0].text(2.8,1.8,'a',size=55)
axes[1].text(3.0,1.8,'b',size=55)
'''
plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)
plt.legend(loc='upper left', frameon=False, fontsize=9)
plt.xlim((0, 3.5))
plt.ylim((0.7, 2.0))
'''

'''
plt.ylabel(r'$a_{c}^{\prime}$', fontsize=17)
plt.legend(loc='upper left', frameon=False, fontsize=9)
plt.xlim((0, 3.5))
plt.ylim((0.7, 2.0))
'''

if anion is 'SCN':
    os.chdir('../../figs')

else:
    os.chdir('../figs')
        
print("Activity derivative error:", actDerErrSimKruger)
print("Current working directory: ", os.getcwd())
fig.tight_layout()
plt.savefig('actDersHess'+str(anion)+'s.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show

In [ ]:
# Calculating and plotting excess coordination numbers and their components:
# CHECK N_c, rho_c, and g_cc - we want to select anions around cations, not salt around salt 

from scipy.signal import argrelextrema
from scipy.interpolate import UnivariateSpline

cations = ['Na']
anions = ['SCN']
rdfIons = ['C']
colors = ['c', 'm', 'b', 'r', 'g', 'y']

string = ''

MW_H2O = 18.01528e-3 # kg/mol

cI = 0
rI = 0
xpos = 0.5
xpos_avg = 0

for anion in anions:
    
    minVal = 10
    
    for cation in cations:
        
        # Analysis
        MW_H2O = 18.01528e-3 # kg/mol
        
        if anion is 'SCN':
            os.chdir(WORKDIR+str(d['Box size'])+'Å/'+cation.lower()+anion.lower())
            print('Current working directory: ', os.getcwd())
            block_range = np.arange(1,19,1)
            
        else:
            os.chdir(WORKDIR+'60Å/'+cation.lower()+anion.lower())
            print('Current working directory: ', os.getcwd())
            block_range = np.arange(1,19,1)

        molar = np.empty([1, len(d['Molar concentrations'])])
        molal = np.empty([1, len(d['Molal concentrations'])])
        actDer = np.empty([len(block_range),len(d['Molal concentrations'])])
        actDerAvg = np.empty([1, len(d['Molal concentrations'])])
        actDerErr = np.empty([2, len(d['Molal concentrations'])])

        index = 0
        chargeSet = 1
        rdfIon = rdfIons[rI]

        for (q_s, q_c, q_n) in zip([d['Partial charge']['S'][chargeSet-1]], [d['Partial charge']['C'][chargeSet-1]], [d['Partial charge']['N'][chargeSet-1]]):

            if anion is 'SCN':
                os.chdir('q_s_'+str(q_s)+'_q_c_'+str(q_c)+'_q_n_'+str(q_n))

            i = 0

            for sigma in [d['Size'][cation]]:

                j = 0

                for eps in [d['Well depth'][cation]]:

                    for conc_m in d['Molal concentrations']:                                
                        print("Current working directory: ", os.getcwd(), "\n")
                        
                        if anion is 'SCN':
                            os.chdir('s'+cation+'_'+str(sigma)+'_e'+cation+'_'+str(eps))
                            os.chdir('conc_'+str(conc_m)+'_'+dirs[i,j])
                            print("Current working directory: ", os.getcwd(), "\n")
                        
                        elif anion is 'I':
                            os.chdir('s'+anion+'_'+str(d['Size'][anion])+'_e'+anion+'_'+str(d['Well depth'][anion]))
                            os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                            print("Current working directory: ", os.getcwd(), "\n")
                            
                        else:
                            os.chdir('conc_'+str(conc_m)+'_sigma_'+cation.lower()+'_'+str(sigma)+'_eps_'+cation.lower()+'_'+str(eps))
                            print("Current working directory:", os.getcwd(), "\n")
                        
                        cnt = 0
                        ind = 0

                        pdb_full = md.load_pdb('out.pdb')

                        N_an = len(pdb_full.topology.select('name '+rdfIon)) # Number of anions
                        N_cat = len(pdb_full.topology.select('name '+cation)) # Number of cations
                        N_c = len(pdb_full.topology.select('name '+rdfIon+' or name '+cation)) # Number of ions
                        N_w = len(pdb_full.topology.select('name O')) # Number of water molecules

                        g_catan_full = 0
                        g_wc_full = 0
                        V_avg = 0

                        for block in block_range:
                            
                            #print("Current working directory:", os.getcwd(), "\n")
                            if os.path.isfile( 'g_+-_'+str(block) ) and os.path.isfile( 'r_'+str(block) ) and os.path.isfile( 'V_'+str(block) ) and os.path.isfile( 'rho_-_'+str(block) ):        # if pickle file exists, use that
                                print("Loading from saved pickle")

                                g_catan = np.loadtxt('g_+-_'+str(block))
                                g_catan_full = g_catan_full+g_catan/len(block_range)
                                #g_wc = np.loadtxt('g_wc_'+str(block))
                                #g_wc_full = g_wc_full+g_wc/len(block_range)
                                r = np.loadtxt('r_'+str(block))
                                V = np.loadtxt('V_'+str(block))
                                V_avg = V_avg + np.loadtxt('V_'+str(block))/len(block_range)
                                rho_an = np.loadtxt('rho_-_'+str(block))

                            else:
                                while (cnt < 1):
                                    print("Reading in trajectory...")
                                    sel = pdb_full.topology.select('name O or name '+cation+' or name '+rdfIon)
                                    traj_full = md.load_dcd('out.dcd', top='out.pdb', atom_indices = sel)
                                    print("Number of atoms = ", traj_full.n_atoms)
                                    nbrFrames = traj_full.n_frames
                                    start = 2000
                                    traj_equil = traj_full[start:nbrFrames]
                                    print("Number of frames in equilibrated trajectory: ", traj_equil.n_frames)
                                    cnt = cnt + 1

                                print('Loading from MC raw data')   # otherwise extract from MC output

                                startFrac = start / (d['Steps']['Simulation'] / d['Steps']['Report'])

                                print('Start fraction: ', startFrac)

                                endFrac = 1 - startFrac
                                first = nbrFrames*startFrac + (block-1)*(nbrFrames*endFrac)/len(block_range)
                                last = nbrFrames*startFrac + block*(nbrFrames*endFrac)/len(block_range)

                                traj = traj_full[first:last]

                                V = 0
                                for vec in traj.unitcell_lengths:
                                    V = V + vec[0]*vec[1]*vec[2] / traj.n_frames

                                V_avg = V_avg + V/len(block_range)

                                rho_an = N_an / V # Average number density of ions (nm⁻³)
                                print("rho_an =",rho_an)
                                rho_w = N_w / V # Average number density of water molecules (nm⁻³)

                                # Analysis: Calculating RDFs
                                r_max = 2.80 #V**(1/3.)/2
                                #print("Half of box side =", V**(1/3.)/2)
                                pair_catan = traj.topology.select_pairs('name '+cation, 'name '+rdfIon)
                                pair_wc = traj.topology.select_pairs('name O', 'name '+rdfIon)
                                r, g_catan = md.compute_rdf(traj, pair_catan, r_range=[0,r_max], bin_width=0.01, periodic=True)
                                r, g_wc = md.compute_rdf(traj, pair_wc, r_range=[0,r_max], bin_width=0.01, periodic=True)

                                g_catan_full = g_catan_full+g_catan/len(block_range)
                                g_wc_full = g_wc_full+g_wc/len(block_range)

                                for (file_name, var) in zip(['g_+-_'+str(block), 'g_w-_'+str(block), 'r_'+str(block), 'V_'+str(block), 'rho_-_'+str(block)], [g_catan, g_wc, r, V, rho_an]):
                                    if file_name in ('V_'+str(block), 'rho_-_'+str(block)):
                                        f = open( file_name, 'w' )
                                        f.write( repr(var) + '\n' )
                                        f.close()
                                        print("check......")
                                    else: 
                                        np.savetxt(file_name, var)
                                        print("next check...")


                            # Calculating coordination numbers
                            dr = r[1]-r[0]
                            print('Length of r and g_catan:', len(r), len(g_catan))
                            N_catan = rho_an * 4 * pi * np.cumsum( ( g_catan - 1 ) * r ** 2 * dr )
                            
                            #N_wc = rho_an * 4 * pi * np.cumsum( ( g_wc - 1 ) * r ** 2 * dr )

                            Vn = 4*pi/3*r**3 / V
                            corr_catan_func = N_an * ( 1 - Vn ) / ( N_an * ( 1 - Vn ) - N_catan - 1 )
                            #corr_wc_func = N_an * ( 1 - Vn ) / ( N_an * ( 1 - Vn ) - N_wc - 0 )
                            corr_catan = np.mean(corr_catan_func[95:115])
                            #corr_wc = np.mean(corr_wc_func[95:115])

                            g_catanc = g_catan * corr_catan
                            #g_wcc = g_wc * corr_wc
                            N_catanc = rho_an * 4 * pi * np.cumsum( ( g_catanc - 1 ) * r ** 2 * dr )
                            #N_wcc = rho_an * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr )

                            ind = ind + 1
                        
                        print("Current working directory: ", os.getcwd())
                        
                        rho_c = N_c / V_avg # Average number density of ions (nm⁻³)
                        rho_an = N_an / V_avg
                        print("rho_- =",rho_an)
                        #rho_w = N_w / V_avg # Average number density of water molecules (nm⁻³)

                        dr = r[1]-r[0]
                        N_catan = rho_an * 4 * pi * np.cumsum( ( g_catan_full - 1 ) * r ** 2 * dr )
                        #N_wc = rho_c * 4 * pi * np.cumsum( ( g_wc_full - 1 ) * r ** 2 * dr )

                        Vn = 4*pi/3*r**3 / V_avg
                        corr_catan_func = N_an * ( 1 - Vn ) / ( N_an * ( 1 - Vn ) - N_catan - 1 )
                        #corr_wc_func = N_an * ( 1 - Vn ) / ( N_an * ( 1 - Vn ) - N_wc - 0 )
                        corr_catan = np.mean(corr_catan_func[95:115])
                        #corr_wc = np.mean(corr_wc_func[95:115])

                        g_catanc = g_catan_full * corr_catan
                        #g_wcc = g_wc_full * corr_wc
                        
                        indCIP = sigma*100
                        print(indCIP)
                        
                        #rspline = np.linspace( 0, len(r)/100, len(r))
                        #spl = UnivariateSpline( r, g_catanc, k=3 )
                        #spl.set_smoothing_factor(0.0009)
                        
                        #print(spl(rspline), rspline)
                        
                        if anion is 'Cl' or anion is 'I':
                            order = 3
                        
                        else:
                            order = 1
                            
                        limits = np.append([0], argrelextrema(g_catanc, np.less, order = order)) # [0, indCIP+5, indCIP+20, indCIP+40, indCIP+60] # Define indexes giving radii between which the minima of the RDF representing the end of the first, second and third solvation shells are found
                        #limits = [0, indCIP+5, indCIP+20, indCIP+40, indCIP+60]
                        #limits = np.append([0], argrelextrema(spl(rspline)[40:150], np.less, order = 5)) # [0, indCIP+5, indCIP+20, indCIP+40, indCIP+60] # Define indexes giving radii between which the minima of the RDF representing the end of the first, second and third solvation shells are found
                        limits = limits[0:4]
                        
                        if cation is 'Na' and anion is 'SCN':
                            limits[3] = 86
                        
                        if cation is 'K' and anion is 'SCN':
                            limits[3] = 95
                        
                        print("Index array: ", limits)
                        limInds = np.arange(0,4,1)

                        bound = np.zeros(len(limInds))
                        r_part = np.zeros(len(limInds)-1)
                        
                        N_catanc_part_plateau = np.zeros(len(limInds)-1)
                        N_catanc_tot = 0
                        
                        print("Indexes:", len(limits))
                        

                        for ind in range(0, len(limits)-1):
                            start = limits[ind]
                            stop = limits[ind+1]
                            print('Example index:', stop)
                            print('Distance from center of molecule:', r[stop], 'nm')
                            N_catanc_part = rho_an * 4 * pi * np.cumsum( ( g_catanc[start:stop] - 1 ) * r[start:stop] ** 2 * dr )
                            N_catanc_part_plateau[ind] = N_catanc_part[-1]
                            N_catanc_tot = N_catanc_tot + N_catanc_part_plateau[ind]

                        print("Excess coordination numbers =", N_catanc_part_plateau)
                        print("Sum of coordination numbers =", N_catanc_tot, "\n")
                        
                        cutoffInd = 100
                        N_catanc = rho_an * 4 * pi * np.cumsum( ( g_catanc[0:cutoffInd] - 1 ) * r[0:cutoffInd] ** 2 * dr )
                        N_catanc = N_catanc[-1]
                        print("Cumulative excess coordination number =", N_catanc, "\n")
                        #N_wcc = rho_an * 4 * pi * np.cumsum( ( g_wcc - 1 ) * r ** 2 * dr )

                        molar[0, index] = N_c/(2*d['Constants']['A']*V_avg*1e-24)
                        molal[0, index] = N_c/(2*N_w*MW_H2O)

                        print("Propertites for", molar[0, index], "M:")
                        print("Number of ions =", N_c)
                        print("Number of anions =", N_an)
                        print("Number of cations =", N_cat)
                        print("Number of water molecules =", N_w)
                        print("Molality =", molal[0,index], "m")
                        print("Average box side length =", V_avg**(1/3), "nm")
                        print("Average box volume =", V_avg, "nm³")

                        index = index + 1

                        os.chdir('..')
                        
                        if anion is 'SCN' or anion is 'I':
                            os.chdir('..')
                        
                    j = j+1
                i = i+1

        print("Current working directory: ", os.getcwd())
        #print("Activity derivatives =", actDer)
        #print("Molalities =", c_m)
        
        ax = plt.subplot(111)
    
        N_catanc_cip = N_catanc_part_plateau[0]
        N_catanc_sip = N_catanc_part_plateau[1]
        N_catanc_2sip = N_catanc_part_plateau[2]

        listN = [N_catanc_cip, N_catanc_sip, N_catanc_2sip, N_catanc]
        
        print(listN)
        
        if minVal > min(listN):
            minVal = min(listN) 
            print("Minimum value:", minVal)
        
        if cation is cations[-1]:
            if minVal > 0:
                print("Minimum value more than than 0")
                minVal = 0
        
        if cation is cations[0]:
            xpos = xpos+0.15
            xpos_avg = xpos
            print("Last anion")
            
        else:
            xpos_avg = xpos
            xpos = xpos+0.065
            xpos_avg = (xpos_avg + xpos)/len(cations)
            print("average x-pos:", xpos_avg)
            
        ypos = max(listN)
        '''
        if anion is anions[-1] and cation is cations[-1]:
            ax.bar(xpos, N_catanc , width=0.045, edgecolor='crimson', color='crimson', label='$\Delta N_{+-}$'  , align='center', alpha = 1.0) 
            ax.bar(xpos, N_catanc_cip , width=0.035, edgecolor='indigo', color='white',  ecolor='indigo', label='$\Delta N_{CIP}$', align='center', hatch = '/////', fill=True, alpha = 1.0) 
            ax.bar(xpos, N_catanc_sip , width=0.025, edgecolor='navy',   color='navy',   label='$\Delta N_{SIP}$' , align='center', alpha = 1.0)
            ax.bar(xpos, N_catanc_2sip, width=0.015, edgecolor='pink',   color='white',  ecolor='pink', label='$\Delta N_{2SIP}$', align='center', hatch = '\\\\', fill=True, alpha = 1.0)
        
        
        else:
            ax.bar(xpos, N_catanc , width=0.045, edgecolor='crimson', color='crimson', align='center', alpha = 1.0) 
            ax.bar(xpos, N_catanc_cip , width=0.035, edgecolor='indigo', color='white',  ecolor='indigo', align='center', hatch = '/////', fill=True, alpha = 1.0) 
            ax.bar(xpos, N_catanc_sip , width=0.025, edgecolor='navy',   color='navy',   align='center', alpha = 1.0)
            ax.bar(xpos, N_catanc_2sip, width=0.015, edgecolor='pink',   color='white',  ecolor='pink', align='center', hatch = '\\\\', fill=True, alpha = 1.0)
        
        ax.axhline(y=0, color='k')
        ax.axvline(x=0, color='k')
        ax.set_xticks([])
        
        ax.text(xpos, ypos + 0.01, cation+'$^+$', ha='center', va='bottom', fontsize=10)
        
    
        
        
        print("Excess coordination number, cip:", N_catanc_cip)
        print("x- and y-positions:", xpos, ypos)
        
        #minVal = np.min(listN)
        
        if cation is cations[-1]:
            ypos = minVal - 0.01
            ax.text(xpos_avg, ypos - 0.06, anion+'$^-$', ha='center', va='bottom', fontsize=10)
            plt.legend(loc=0, frameon=False, fontsize=10)
            plt.ylabel('$\Delta N$')

        print("Average x-position: ",xpos_avg)
        
        plt.axis([0.5,1.5,-0.2,0.6])
        '''
        '''
        if anion is anions[-1] and cation is cations[-1]:
            

            
            plt.savefig('excCoordNbrs', bbox_inches='tight', pad_inches=0.1)


            plt.figure()
            plt.xlabel('$r$ [nm]')
            plt.ylabel('$g(r)$')
            plt.plot(r, g_catanc, 'm', label=anion+' around '+cation)
            #plt.plot(rspline, spl(rspline), 'r-', label=anion+' around '+cation+' splined')
            plt.legend(loc=0, frameon=False, fontsize=14)
            plt.axis([0,1.5,0,5])
            plt.savefig('rdf_'+str(anion)+'_'+str(cation), bbox_inches='tight', pad_inches=0.1)
            #plt.ylim(1.0,1.15)
            #plt.xlim(0.8,1.2)
            
        ''' 
        cI = cI + 1
        
        os.chdir('..')

    rI = rI + 1

plt.savefig('rdf_'+str(anion)+'_'+str(cation)+'.png', bbox_inches='tight', pad_inches=0.1)
#plt.savefig('paramSetAllSalts', bbox_inches='tight', pad_inches=0)
plt.show()
print("Current working directory: ", os.getcwd())

In [ ]:
plt.plot(r, corr_wc, 'k-')
plt.plot(r, corr_cc, 'g-')

In [ ]:
# Plotting distributions of bondlengths in SCN⁻ 

print("Current working directory: ", os.getcwd())
os.chdir('paramsFromJana/40ns/chargesFromDFT')
for sigma_s in [0.35]:
        for eps_s in [1.55]:
            for conc in conc_range:
                os.chdir('conc_'+str(conc)+'_sigma_s_'+str(sigma_s)+'_eps_s_'+str(eps_s))
                traj_full = md.load_dcd('out.dcd', top='out.pdb')
                nbrFrames = traj_full.n_frames
                index = index + 1
                ind = -1
                
                
                sel = ['s', 'n']
                bond_lengths = [l_sc, l_cn]
                
                for i, atom in enumerate(sel):
                    bond_length = np.empty(0)
                    for res in traj_full.topology.residues:
                        if (res.name == 'SCN'):
                            other_atom = traj_full.topology.select('resid '+str(res.index)+' and name '+str(atom.upper()))

                            c = traj_full.topology.select('resid '+str(res.index)+' and name C')

                            pair = traj_full.topology.select_pairs(other_atom,c)
                            dist = md.compute_distances(traj_full,pair)
                            dist = dist.reshape(dist.size)
                            bond_length = np.append(bond_length,dist)
                                              
                    x = np.arange(bond_lengths[i]-0.02,bond_lengths[i]+0.02,0.00005)
                    hist_c,b = np.histogram(bond_length,bins=x,density=True)
                    plt.figure()
                    plt.plot(b[:-1],hist_c,lw=3,color='black')
                    plt.xlabel('C-'+str(atom.upper())+' / nm')
                    plt.ylabel('P(C-'+str(atom.upper())+')')
os.chdir('../..')

In [ ]:
# Generating snapshots used for DFT calculations 

import os
#os.chdir('../../../../..')
print("Current working directory: ", os.getcwd())
#os.chdir('nobackup/users/eko12vas/scn')

simTimes = ['2ns', '5ns', '10ns', '20ns', '40ns']

conc = [0.45, 0.78, 1.08, 1.71] #0.47
sigma_s = 0.35 #0.38
eps_s = 1.5 #1.5
os.chdir('paramsFromJana/'+str(simTimes[4])+'/chargesFromPetersenSaykally/conc_'+str(conc[0])+'_sigma_s_'+str(sigma_s)+'_eps_s_'+str(eps_s))
filename = 'out.dcd'
import pytraj as pt
t = pt.Trajectory(filename,top='out.pdb')
t.topology.set_solvent(':HOH')
os.chdir('../../../..')
print('Current working directory =', os.getcwd(), "\n")


In [ ]:
# Generating snapshots used for DFT calculations 

import numpy as np
#os.chdir('nobackup/users/eko12vas/scn')
#os.chdir('nobackup/users/eko12vas/scn')
#os.chdir('../../../..')

print("Current working directory: ", os.getcwd())
os.chdir('paramsFromJana/'+str(simTimes[4])+'/chargesFromPetersenSaykally/conc_'+str(conc[0])+'_sigma_s_'+str(sigma_s)+'_eps_s_'+str(eps_s)+'/snapshots')


nbrSnapshots = 17

for k in [30]:  # number of solvent molecules selected
    l=0
    for j in range(t.n_frames):
        m=j*10
        for i in np.arange(1,nbrSnapshots+1, 1):

            #print(t.topology.n_solvents)
            coord = t[m,':'+str(i)].xyz
            #print(coord)
 
            if ((10 < coord[1,0] < 30) and (10 < coord[1,1] < 30) and (10 < coord[1,2] < 30)):
                
                l = l + 1
                print("#",l,"set of accepted coordinates:", coord[1,:])
            
                t_shell = pt.closest(t[m,':'+str(i)+',HOH'], mask=':SCN', n_solvents=k, top=t[':'+str(i)+',HOH'].top, dtype='trajectory')
                pt.principal_axes(t_shell, mask=':SCN', dorotation=True)
                t_shell = pt.center(t_shell, mask=':SCN', center='box')
                pt.write_traj('shell_'+str(k)+'_'+str(l)+'.pdb',traj=t_shell,top=t_shell.top,overwrite=True,options='model')
                ss = md.load_pdb('shell_'+str(k)+'_'+str(l)+'.pdb')
                ss.save_xyz('shell_'+str(k)+'_'+str(l)+'.xyz')


                if l == 50:
                    break
            if l == 50:
                break
        if l == 50:
            break
            
os.chdir('../../../../..')

print('Current working directory =', os.getcwd(), "\n")

In [ ]:
# Visualizing snapshots

import nglview as nv

#os.chdir('nobackup/users/eko12vas/scn')
print("Current working directory: ", os.getcwd())
#os.chdir('../../../..')
os.chdir('paramsFromJana/'+str(simTimes[4])+'/chargesFromPetersenSaykally/conc_'+str(conc[0])+'_sigma_s_'+str(sigma_s)+'_eps_s_'+str(eps_s)+'/snapshots')

print('Current working directory =', os.getcwd(), "\n")

nbrSolvSel = 30

t = pt.Trajectory('shell_'+str(nbrSolvSel)+'_*.pdb','shell_'+str(nbrSolvSel)+'_1.pdb')
t = pt.center(t, mask=':SCN', center='box') # with mask=:SCN, selection is not centered
pt.principal_axes(t_shell, mask=':SCN', dorotation=True)

#print('Current working directory =', os.getcwd(), "\n")

view = nv.show_pytraj(t)

view.add_representation('licorice')
os.chdir('../../../../..')
print("Current working directory: ", os.getcwd())

view

# References

A Refined All-Atom Potential for Imidazolium-Based Room Temperature Ionic Liquids: Acetate, Dicyanamide, and Thiocyanate Anions 
http://pubs.acs.org/doi/abs/10.1021/acs.jpcb.5b02272

Solvation of Ln(III) Lanthanide Cations in the [BMI][SCN], [MeBu3N][SCN], and [BMI]5[Ln(NCS)8] Ionic Liquids: A Molecular Dynamics Study
http://pubs.acs.org/doi/abs/10.1021/ic802227p

Enhanced Concentration of Polarizable Anions at the Liquid Water Surface:  SHG Spectroscopy and MD Simulations of Sodium Thiocyanide
http://pubs.acs.org/doi/abs/10.1021/jp050864c

A Kirkwood-Buff Derived Forcefield for Aqueous Alkali Halides
http://pubs.acs.org/doi/abs/10.1021/ct100517z